In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_shenghuo_richang=pd.DataFrame(columns=columns)
channel_shenghuo_richang

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[9]/div/a/span').click()  #选择“生活”频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[9]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'生活'

In [7]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[3]/a').click()  #选择“日常”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[3]/a').text
sub_channel

'日常'

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [12]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_shenghuo_richang.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-22 13:17:03.398299


<ipython-input-12-815cb9756d3a>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-12-815cb9756d3a>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-12-815cb9756d3a>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1EJ411n7Dg视频属性的抓取
BV号为1EJ411n7Dg的视频属性抓取结束


<ipython-input-12-815cb9756d3a>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1Z7411Y7Jx视频属性的抓取
BV号为1Z7411Y7Jx的视频属性抓取结束
开始BV号为1sJ411j7MP视频属性的抓取
BV号为1sJ411j7MP的视频属性抓取结束
开始BV号为1X7411e74h视频属性的抓取
BV号为1X7411e74h的视频属性抓取结束
开始BV号为117411i7Q4视频属性的抓取
BV号为117411i7Q4的视频属性抓取结束
开始BV号为1z7411k7wy视频属性的抓取
BV号为1z7411k7wy的视频属性抓取结束
开始BV号为1A7411e71Y视频属性的抓取
开始BV号为1A7411e71Y视频属性的抓取
BV号为1A7411e71Y的视频属性抓取结束
开始BV号为147411i7EM视频属性的抓取
BV号为147411i7EM的视频属性抓取结束
开始BV号为1H7411Y7kG视频属性的抓取
BV号为1H7411Y7kG的视频属性抓取结束
开始BV号为1vJ411H7mb视频属性的抓取
BV号为1vJ411H7mb的视频属性抓取结束
开始BV号为1AJ411V7Ha视频属性的抓取
BV号为1AJ411V7Ha的视频属性抓取结束
开始BV号为1Q7411Y7YM视频属性的抓取
BV号为1Q7411Y7YM的视频属性抓取结束
开始BV号为157411k7q9视频属性的抓取
BV号为157411k7q9的视频属性抓取结束
开始BV号为1QJ411L7yM视频属性的抓取
BV号为1QJ411L7yM的视频属性抓取结束
开始BV号为1DJ411J7yQ视频属性的抓取
BV号为1DJ411J7yQ的视频属性抓取结束
开始BV号为147411i7Wu视频属性的抓取
BV号为147411i7Wu的视频属性抓取结束
开始BV号为1i7411e7hP视频属性的抓取
BV号为1i7411e7hP的视频属性抓取结束
开始BV号为1s7411k7Nt视频属性的抓取
BV号为1s7411k7Nt的视频属性抓取结束
开始BV号为1R7411q7PC视频属性的抓取
BV号为1R7411q7PC的视频属性抓取结束
开始BV号为1hJ411H7MH视频属性的抓取
BV号为1hJ411H7MH的视频属性抓取结束
开始BV号为1yJ411j7vA视频属性的抓取
BV号为1yJ411j7vA的视频属性抓取结束
开始BV号为1sJ411L74e

...

BV号为1sK411s7RW的视频属性抓取结束
开始BV号为1yz4y1R75x视频属性的抓取
BV号为1yz4y1R75x的视频属性抓取结束
开始BV号为1fA411i7sR视频属性的抓取
BV号为1fA411i7sR的视频属性抓取结束
开始BV号为1Q5411s7YQ视频属性的抓取
BV号为1Q5411s7YQ的视频属性抓取结束
开始BV号为1854y1D7nH视频属性的抓取
BV号为1854y1D7nH的视频属性抓取结束
开始BV号为18V411C7QA视频属性的抓取
BV号为18V411C7QA的视频属性抓取结束
开始BV号为1Vf4y1U775视频属性的抓取
BV号为1Vf4y1U775的视频属性抓取结束
开始BV号为1BV411C72Z视频属性的抓取
BV号为1BV411C72Z的视频属性抓取结束
开始BV号为1JQ4y1P7vM视频属性的抓取
BV号为1JQ4y1P7vM的视频属性抓取结束
开始BV号为1cz411B7mN视频属性的抓取
BV号为1cz411B7mN的视频属性抓取结束
开始BV号为1ra4y1e7PF视频属性的抓取
BV号为1ra4y1e7PF的视频属性抓取结束
开始BV号为1Gf4y1272j视频属性的抓取
BV号为1Gf4y1272j的视频属性抓取结束
开始BV号为1xe411W7gA视频属性的抓取
BV号为1xe411W7gA的视频属性抓取结束
开始BV号为18K4y1t7Tb视频属性的抓取
BV号为18K4y1t7Tb的视频属性抓取结束
开始BV号为1aA411q7fm视频属性的抓取
BV号为1aA411q7fm的视频属性抓取结束
开始BV号为1vK4y1t7qp视频属性的抓取
BV号为1vK4y1t7qp的视频属性抓取结束
开始BV号为1Di4y147oQ视频属性的抓取
BV号为1Di4y147oQ的视频属性抓取结束
开始BV号为1vt4y1C7bZ视频属性的抓取
BV号为1vt4y1C7bZ的视频属性抓取结束
开始BV号为1nA411q7CN视频属性的抓取
BV号为1nA411q7CN的视频属性抓取结束
开始BV号为1fz4y1X72Q视频属性的抓取
BV号为1fz4y1X72Q的视频属性抓取结束
开始BV号为1rz411v798视频属性的抓取
BV号为1rz411v798的视频属性抓取结束
开始BV号为1fK411W7JV

<ipython-input-12-815cb9756d3a>:187: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1T54y1i7ny视频属性的抓取
BV号为1T54y1i7ny的视频属性抓取结束
开始BV号为18K411J7gj视频属性的抓取
BV号为18K411J7gj的视频属性抓取结束
开始BV号为1654y1S7kC视频属性的抓取
BV号为1654y1S7kC的视频属性抓取结束
开始BV号为1FV411z7SP视频属性的抓取
BV号为1FV411z7SP的视频属性抓取结束
开始BV号为1sp4y1i7Ey视频属性的抓取
BV号为1sp4y1i7Ey的视频属性抓取结束
开始BV号为1SV411B7p3视频属性的抓取
BV号为1SV411B7p3的视频属性抓取结束
开始BV号为1qK4y1e7KC视频属性的抓取
BV号为1qK4y1e7KC的视频属性抓取结束
开始BV号为1Zz4y1D7Rk视频属性的抓取
BV号为1Zz4y1D7Rk的视频属性抓取结束
开始BV号为18p4y1q7eo视频属性的抓取
BV号为18p4y1q7eo的视频属性抓取结束
开始BV号为1gV411z7Nm视频属性的抓取
BV号为1gV411z7Nm的视频属性抓取结束
开始BV号为1zV411z7HA视频属性的抓取
BV号为1zV411z7HA的视频属性抓取结束
开始BV号为1dk4y1273o视频属性的抓取
BV号为1dk4y1273o的视频属性抓取结束
开始BV号为1zV411z7HA视频属性的抓取
BV号为1zV411z7HA的视频属性抓取结束
开始BV号为1dk4y1273o视频属性的抓取
开始BV号为1dk4y1273o视频属性的抓取
BV号为1dk4y1273o的视频属性抓取结束
开始BV号为19h411o7zs视频属性的抓取
BV号为19h411o7zs的视频属性抓取结束
开始BV号为1nK4y1x7Cb视频属性的抓取
BV号为1nK4y1x7Cb的视频属性抓取结束
开始BV号为1MK411J7ei视频属性的抓取
BV号为1MK411J7ei的视频属性抓取结束
开始BV号为1qh411Z7t4视频属性的抓取
BV号为1qh411Z7t4的视频属性抓取结束
开始BV号为1pK411H7AA视频属性的抓取
BV号为1pK411H7AA的视频属性抓取结束
开始BV号为1u5411a7JV视频属性的抓取
BV号为1u5411a7JV的视频属性抓取结束
开始BV号为1yK4y1e7JU

BV号为1ND4y1m7rx的视频属性抓取结束
开始BV号为1e54y1y7z1视频属性的抓取
BV号为1e54y1y7z1的视频属性抓取结束
开始BV号为1Hp4y1e7aE视频属性的抓取
BV号为1Hp4y1e7aE的视频属性抓取结束
开始BV号为1bK4y1a7MM视频属性的抓取
BV号为1bK4y1a7MM的视频属性抓取结束
开始BV号为1M64y1F7vy视频属性的抓取
BV号为1M64y1F7vy的视频属性抓取结束
开始BV号为16a4y1j7xz视频属性的抓取
BV号为16a4y1j7xz的视频属性抓取结束
开始BV号为1st4y1q7gv视频属性的抓取
BV号为1st4y1q7gv的视频属性抓取结束
开始BV号为1BD4y1o7rZ视频属性的抓取
BV号为1BD4y1o7rZ的视频属性抓取结束
开始BV号为1YV411m7dX视频属性的抓取
BV号为1YV411m7dX的视频属性抓取结束
开始BV号为1xh411X7TJ视频属性的抓取
BV号为1xh411X7TJ的视频属性抓取结束
开始BV号为1sz4y1Z7Jq视频属性的抓取
BV号为1sz4y1Z7Jq的视频属性抓取结束
开始BV号为1oi4y177oL视频属性的抓取
BV号为1oi4y177oL的视频属性抓取结束
开始BV号为18y4y1k7eE视频属性的抓取
BV号为18y4y1k7eE的视频属性抓取结束
开始BV号为1Sa4y1j7Uo视频属性的抓取
BV号为1Sa4y1j7Uo的视频属性抓取结束
开始BV号为1hp4y1a7hd视频属性的抓取
BV号为1hp4y1a7hd的视频属性抓取结束
开始BV号为1754y117du视频属性的抓取
BV号为1754y117du的视频属性抓取结束
开始BV号为1r54y1y7uu视频属性的抓取
BV号为1r54y1y7uu的视频属性抓取结束
开始BV号为1sk4y117s9视频属性的抓取
BV号为1sk4y117s9的视频属性抓取结束
开始BV号为1nV411m7cS视频属性的抓取
BV号为1nV411m7cS的视频属性抓取结束
开始BV号为1EV411m7mp视频属性的抓取
BV号为1EV411m7mp的视频属性抓取结束
开始BV号为1cZ4y1K757视频属性的抓取
BV号为1cZ4y1K757的视频属性抓取结束
开始BV号为1EV411m7mp

开始BV号为1pf4y1B7ws视频属性的抓取
BV号为1pf4y1B7ws的视频属性抓取结束
开始BV号为1PA411j7CG视频属性的抓取
BV号为1PA411j7CG的视频属性抓取结束
开始BV号为1J5411L7v5视频属性的抓取
BV号为1J5411L7v5的视频属性抓取结束
开始BV号为1Qp4y1k7ie视频属性的抓取
BV号为1Qp4y1k7ie的视频属性抓取结束
开始BV号为1iz4y1o7TN视频属性的抓取
BV号为1iz4y1o7TN的视频属性抓取结束
开始BV号为1fA411L7ub视频属性的抓取
BV号为1fA411L7ub的视频属性抓取结束
开始BV号为11y4y1r7kr视频属性的抓取
BV号为11y4y1r7kr的视频属性抓取结束
开始BV号为1dK4y177GQ视频属性的抓取
BV号为1dK4y177GQ的视频属性抓取结束
开始BV号为1Mi4y177pc视频属性的抓取
BV号为1Mi4y177pc的视频属性抓取结束
开始BV号为1ya4y1L7qP视频属性的抓取
BV号为1ya4y1L7qP的视频属性抓取结束
开始BV号为1kr4y1c7Cj视频属性的抓取
BV号为1kr4y1c7Cj的视频属性抓取结束
开始BV号为1iz4y1C7Zp视频属性的抓取
BV号为1iz4y1C7Zp的视频属性抓取结束
开始BV号为1LT4y1F7Qa视频属性的抓取
BV号为1LT4y1F7Qa的视频属性抓取结束
开始BV号为1JZ4y1V7pd视频属性的抓取
BV号为1JZ4y1V7pd的视频属性抓取结束
开始BV号为1Xz4y1o7Xc视频属性的抓取
BV号为1Xz4y1o7Xc的视频属性抓取结束
开始BV号为14y4y1r7a3视频属性的抓取
BV号为14y4y1r7a3的视频属性抓取结束
开始BV号为1T54y1r7d4视频属性的抓取
BV号为1T54y1r7d4的视频属性抓取结束
开始BV号为1rv41167pT视频属性的抓取
BV号为1rv41167pT的视频属性抓取结束
开始BV号为1pA411E7sq视频属性的抓取
BV号为1pA411E7sq的视频属性抓取结束
开始BV号为1gy4y187Jr视频属性的抓取
BV号为1gy4y187Jr的视频属性抓取结束
开始BV号为1KV411a7VW视频属性的抓取
BV号为1KV411a7VW的视

In [11]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-11-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [13]:
channel_shenghuo_richang.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
1月,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96
2月,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97
3月,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98
4月,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97
5月,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97
8月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99


In [15]:
channel_shenghuo_richang.to_json('channel_shenghuo_richang.json')

In [14]:
channel_shenghuo_richang

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1EJ411n7Dg,1月,2020-01-12 16:02:44,火线妹,个人认证,50.5万,42,本 尊 出 现,生活,日常,普通视频,...,"[生活, 日常, 萌妹子]",00:10,2214.1万,20.1万,4.8万,8.5万,2.5万,29780,5.5万,是
1Z7411Y7Jx,1月,2020-01-21 19:58:33,酚妥拉明白的医路日常,无,2.6万,54,他师傅第一次骗他，也是最后一次骗他,生活,日常,活动作品,...,"[生活, 日常, 正能量, 中国军人, 医生, 感人, 全能打卡挑战, 家人]",03:29,1210.4万,122.7万,99.3万,35.3万,2.8万,11742,8.2万,否
1sJ411j7MP,1月,2020-01-03 19:30:34,顺子Chloe,个人认证,25万,79,我被200w+博主抄袭，还被恶意剪辑变低俗恶趣味，不尊重女性，我们原创作者该怎么办？,生活,日常,活动作品,...,"[校园vlog, vlog, 生活, 日常, 反抄袭, 抄袭, 全能打卡挑战, 学生党, 抖...",04:25,1159.8万,124.9万,33.0万,8.6万,7637,22311,5.0万,否
1X7411e74h,1月,2020-01-23 12:00:03,机智的党妹,个人认证,659.9万,174,【党妹】花21万拍视频可以收获什么？翻跳《寄明月》给大家拜年啦！,生活,日常,活动作品,...,"[寄明月, 中国风, 汉服, 生活, 日常, 中国华服日, 舞蹈, 机智的党妹, 新年焕新妆...",15:32,1075.0万,129.5万,167.9万,49.1万,9.4万,21934,10.5万,是
117411i7Q4,1月,2020-01-18 16:47:17,洗剪吹六块钱,无,2.2万,414,咏 春 打 狗,生活,日常,普通视频,...,"[叶问, 柯基, 功夫, 搞笑, 生活, 日常, 搞笑视频, 咏春拳, 武术]",00:20,1060.7万,32.5万,4.5万,15.3万,12.0万,6152,4177,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1ea411A7yq,10月,2020-10-25 00:25:25,陈师姬,无,14.5万,3916,曹植要是有这三步早跑了,生活,日常,普通视频,...,"[跑步, 科普, 生活, 日常, 生活记录, 曹植, 三国, 高能]",00:24,122.8万,6.2万,676,7339,2258,905,131,否
16a4y1L7Y3,10月,2020-10-21 18:22:11,YouTube精彩视频-,个人认证,45.4万,1001,视力不好的人眼里的世界，太可怕了~,生活,日常,普通视频,...,"[生活, 日常, 视力, 近视眼, 近视, 眼睛]",02:29,122.6万,5.4万,4741,2.5万,1.3万,3093,3590,否
1uD4y197T6,10月,2020-10-30 23:24:59,卧龙寺,无,77.5万,2.6万,男子被捕后爽到不行，原因把警察都逗笑了,生活,日常,普通视频,...,"[生活, 日常, 生活, 记录]",00:18,121.4万,5.4万,326,5786,1485,440,82,否
1BD4y1978K,10月,2020-10-25 10:20:04,-______嘿______-,无,2896,397,兄弟，说好的男女混浴呢？,生活,日常,普通视频,...,"[生活, 日常, 女神, 小视频, 记录, 男人, 洗澡, 沙雕视频]",00:12,121.3万,1.3万,56,959,1067,489,156,否
